# Object Detection and Segmentation

Other than the typical classification related usecases, some of the main other vision use cases are object detection and segmentation. Object detection is identifying the objects in a image. Segmentation means separating out image elements. This can be instance level separation or whole image separation. These 2 types are called instance segmentation and whole-scene semantic segmentation.

## Object detection

Unlike image classification, where we only try to detect whether there's an object, here we try to locate that object in the image. But as you we can already see, object classification already need to locate the object before classification. Problem is those algorithms does not return the object location information. So to make use of that, easiest solution is to predict bounding boxes arorund the detected object. That's in fact the idea behind 'YOLO' approach. 

Unfortunately using typical network architectures does not yield the best result for object detection. Therefore researchers introduced a new architecture called `Feature Pyramid Networks or FPNs` and illustrated their usage with RetinaNet.

---

### **YOLO (You Only Look Once)**

This is one of the simplest object detection architectures with fastest prediction times. Due to this reason it is used in many real time applications. Also this has now been evolved in to many other versions like YOLO v2 - v5 as well with slight differences to the architecture for better prediction result and performance improvements.

Basic idea behind YOLO is that it divides an image to NxM cells and tries to predict a bounding box for an object that may be in those cells. Bounding box may not necessarilly be same size as the initial cell. But predicted box center should be inside of the considering cell. 

<center><image src="./imgs/8.png" width="400px"/></center>

Then for each of these grid cell, YOLO will output below values.

    x: center x of the detected object in that cell (tanh activation)
    y: center y of the detected object in that cell (tanh activation)
    w: width of the detected object as ratio (sigmoid activation)
    h: width of the detected object as ratio (sigmoid activation)
    c: confidence for the detected object
    class: this is a softmax for the classes we train our model

<center><image src="./imgs/9.png" width="600px"/></center>

Note in the above image feature map is for a single data input (For a single image). So single vertical column depicts the outputs related to single grid cell.

As you can see due to this reason, it is important to make sure that final feature map have correct dimentions so that we can get the desired representation of output. One way to achieve this is carefully design the ML network to get the required dimensions. But typically this is cumbersome to achieve. Instead authors suggested to flatten the 3D feature map and then send it through a fully connected layer with size (grid_w x grid_h x (5 + numclasses)) and then reshape it to our requirement. Apparently this yields better results compared to directly using the feature map.

To build the loss function for this we need training data with ground truth boxes and their classes. So then we can compare our output predicted boxes against the ground truth boxes. Also loss function need to consider classification results as well. This is straight forward if a grid cell predicts a single box. But this is not the case in practice. Because most of the time there may be cases some objects may space across multiple grid cells making them overlap. In these cases having one prediction box per cell is not enough. To solve this problem, in YOLO architecture there's a parameter to define number of bounding boxes per a grid cell. This means in the output there will be more x,y,w,h,c values. Check the below image.

<center><image src="./imgs/10.png" width="400px"/></center>

So with these new additions, comparing the prediction with ground truth becomes bit more complex. In fact, instead of just directly comparing each prediction with the ground truth, YOLO uses a metric called 'IOU' or Intersection over Union between all the ground truth boxes and all predicted boxes within a grid cell. Then it select the pairing with highest IOU as the correct prediction.

<center><image src="./imgs/11.png" width="400px"/></center>

With this in place, Total loss per a sample would be a combination of below values.

1. **Object Presence Loss:**
        Each cell that has a ground truth box, compute (1-C)^2. (Loss for not correctly predicting the object presence).
2. **Object Absence Loss:**
        Each cell that does not have a ground truth box computes C^2. (Loss for predicting a object which is not available)
3. **Object classification Loss:**
        Typical cross entropy loss (Loss for incorrectly classifying the detected object).
4. **Bounding Box Loss**
        Loss for bad bounding box dimension prediction.

Then YOLO combine all above losses with different weighting factors to optimize the model for object detection.

One of the  main limitation of the YOLO architecture is that it can only predict single class per grid cell(can select multiple boxes per grid cells though). So will not work if there are multilple items in single grid cell.

Another issue is grid itself. This grid cell separation makes the model not invariant to scale. So unless we carefully tune the model, it will not be able to identify small objects.

Also YOLO tends to locate objects with low precision. This may be because YOLO tries to identify the bounding box from the last feature map of the network. (Usually at such level, spacial details of the original image tend to not be in the last feature map)

---

### **RetinaNet**

Compared to the YOLOv1, this has several new concepts used in its architecture and in its loss calculation. Their architecture design include feature pyramid networks which combine information extracted in multiple scales as well. 

#### **Feature Pyramid Maps**


In the Yolo architecture, detection part only used the last high level feature maps that comes after all the processings of the convolutional layers. This causes the model to be less accurate in its localization capability as the high level features have less spacial resolution by design. 

To circumvent this problem, researchers looked in to a way to combine early level feature maps into detection head inputs hoping that would help to improve the localization capabilities. Thus feature pyramid networks were born. Check the below diagram.

<center><image src="./imgs/12.png" width="500px"/></center>

As we can see in Feature pyramid networks high level feature maps get upsampled and transfered back to low level. Then such combined feature maps will be used in detection tasks. 

<center><image src="./imgs/13.png" width="500px"/></center>

In the FPN, high level feature maps get upsampled and then get added up to the low level feature maps. This helps the spacial features to get preserved while having fine tuned fearures. Then such combined feature maps will get send to the detection heads to produce the detection box and classification. The point to note is that, channel size of inputs to the detection head. Since all these inputs get fed into same (shared) detection head it is important to have same depth size. (Other dimensions can be managed because of the convolutional operations)

Also FPN does not depend on the convolutional network it is being used as long as it can access the intermediate feature maps. So it can be used with any major networks like Resnets or Efficient nets. 


#### **Anchor Boxes**

In YOLO architecture, detections were happened considering a predefined base grid. and trying to minimize that grid to fit the objects. Instead some architectures expanded this idea by predefining a set of 'anchor boxes' with various aspect ratios and scales. Then these boxes were used as the bases for the detection boxes. Idea behind this is to make the model to predict values around zero (Since objects can be in various places anchor boxes only need to have small adjustments compared to grids). This helps neural network activation function to be more actively predict the output values. But this would also face some of the problems encountered in the YOLO grid based detection (like more objects than the boxes etc). So this value need to be setted up based on the requirement and need to be in various sizes and aspect ratios for best results.

In the context of RetinaNet there are nine types of anchor box types in various scales and aspect ratios. Those along with the feature maps computed by the feature pyramid network become the inputs to the final prediction tasks. In summary RetinaNet sequence of operations for predicting bounding boxes are as follows.

1. Feature pyramid network reduce the image in to feature maps.
2. Each map will be used to predict the bounding boxes related to the anchors. For example 4 * 6 (* 256 channels) feature map will have 24 (4 * 6) anchor locations.

<center><image src="./imgs/14.png" width="400px"/></center>

3. The detection head part uses multiple convolutional layers to convert the input feature maps to final output which should have (num_anchor_boxes_per_location * 4) channels. Those additional 4 values per box provides the deltas to center(x, y), width and height of predicted bounding box compared to the anchor box.

4. Like wise each feature map from Pyramid network will use different scales of anchor boxes to predict bounding boxes. 

This different scaling of anchor boxes in each feature pyramid levels should be harmonize with the input image size. For example in RetinaNet there are 5 feature maps (p1, p2, p3, p4, p5) and for each feture map 2^n times small anchor boxes used compared to the input image where n is the feature map depth (eg:- p3 feature map --> n=3).

> Due to above mentioned reasons it is important to finetune the anchor box sizes and scales or resize input images to match the corresponding detection tasks.

When calculating the detection loss, we need to take different approach compared to early models. But the generic loss calculation ideas remain the same. Before the loss calculation we first need to assign ground truth boxes to best anchor.

To do that, we assign the anchor boxes to the ground truth boxes which have the highest IOU value.So we first calculate all the pairwise IOU between anchors and ground truths. Then we arrange those in a matrix format with N (number of ground truth boxes) rows and M (number of anchor boxes) as columns.

Then this matrix is analyzed by columns under below conditions.

- an anchor is assigned to ground truth that has the largest IOU in its column provided it is greater than 0.5. 
- IOU should be greater than 0.4 or it is considered as detected nothing.

Once ground truths are paired with anchors, model can go ahead with the computing box predictions, classification and losses.

#### Architecture

The outputs(Feature maps) from the FPNs are used by the detection head and classification head to predict the class of the object and it's bounding box deltas. As mentioned above feature maps are 3 dimensional where their x, y are spacial and other being number of channels.

In RetinaNet for every spacial location for every feature map several parameters will get predicted.

B: number of anchor box types (Retina net this is 9)
K: number of classes

1. The prediction head predicts the class probabilities (B * K), for every anchor point.
2. The detection head predicts the (B * 4) bounding box deltas (explained in previous section). 

These 2 heads have same type of design and will evaluate feature maps from all the scales sent from the FPN.

<center><image src="./imgs/15.png" width="200px"/></center>



#### Focal Loss for Classification

When we think about the number of anchor points for a input image it get adds up to considerable level of number considering we need to have anchor points for several levels of feature maps. Also we need to have multiple anchor boxes per anchor point as well. This causes large number of boxes to do predictions. But in practice one input image would only have very small amount of ground truth boxes.

This creates a problem when calculating the error. Since there are so many background detecting anchor boxes, errors calculated from those can reduce the effectiveness of the boxes that detected the actual objects. To fix this issue, RetinaNet Paper suggested an impressive solution. They tweaked the loss function to produce much smaller values on empty backgrounds. This technique is called "Focal Loss".

As we can see in the image provided in the above section, classification head uses sigmoid functions to calculate the final outputs. So for all classes they used the binary cross entropy losses. The loss formula for single class predicted probability p, actual label value y would be like below.

`Cross Entropy(y, p) = − y ⋅log(p) − (1 − y)⋅log(1 − p)`

The modification of above to get the Focal loss formula is as follows.

`Focal Loss(y, p) = − y ⋅ (1 − p)^γ ⋅log(p) − (1 − y) ⋅p^γ ⋅log(1 − p)`

When the parameter gamma (γ) is 0, this get simplfied to the base cross entropy formula. But in other case it's behaviour is bit different. In the case of y==0 (which means ground truth background) if we plot the Focal loss to the probability values for various gamma values we will be able to see that the loss is getting exponentially reduced to lower levels. This way model can safely add up errors from all the anchor boxes and still train the model to perform better. (And another reason why our mathematic knowledge should be very good. We can utilize these kinds of cool tricks to solve our problems!)


#### Smooth L1 loss for Bounding box error

The detection head does regression as its objective is to provide values for the delta values. The normal way of calculating the error in regression tasks is L1(absolute error) and L2(squared error) loss. Both of these have their own strengths and weaknesses, so usage need to be thought accordingly. But in the case of RetinaNet authors choosed the middle ground of both which is called `Huber Loss` or `Smooth L1 loss`.

This loss calculation method behaves like L2 loss for small values and L1 loss to large values. This helps the model to unnecessarily generate large errors while maintaining good convergance ability.

#### Non Maximum Suppression

As you might(should) probably wondering, since many anchor boxes can be on top of a detected object, once all the classifications and identifications done there can be multiple bounding boxes per single identified object. 

<center><image src="./imgs/16.png" width="200px"/></center>

To fix that issue we can use the technique `Non Maximum Suppression`. Here what happens is it takes the detected box IOU and its confidence to find the most appropriate bounding box. Its very simple to implement without any optimizations like below.

In [1]:
# JUST A PSUEDO CODE

def NMS(boxes, class_probabilities_per_box, IOU_THRES = 0.5):
    '''
        This only consider bounding box for a single class
    '''

    result_boxes = []

    for b1 in boxes:
        non_useful = False

        for b2 in boxes:
            if (IOU(b1, b2) > IOU_THRES):
                if(class_probabilities_per_box[b2]>class_probabilities_per_box[b1]):
                    non_useful = True
        if(not non_useful):
            result_boxes.append(b1)

    return result_boxes

This is pretty simple logic which is very useful in practice, but can have some unwanted problems as well. One example is if there are 2 objects in close range which got detected, but have a overlapping IOU more than the defined threshold, then the object with lower confidence will get ignored. To fix such issues there's a variant of NMS called `Soft NMS` which continuously decrease the confidence of overlapping bounding boxes. This helps to reduce the scenarios of objects getting missed.